## Day 5 Session 1

It Consist's of all the programs from Day 5 Session 1.

In [6]:
import requests
import json  # Used to pretty-print JSON

print("--- Example 1: Basic GET Request ---")

# Example 1: Get a single "todo" item from a public test API
url_todo_item = "https://jsonplaceholder.typicode.com/todos/1"
print(f"Trying to get data from: {url_todo_item}")

try:
    response = requests.get(url_todo_item)  # Send GET request
    print(f"Status Code: {response.status_code}")  # 200 means success

    # Convert JSON response to a Python dictionary
    todo_item = response.json()
    print("\nResponse as Python Dictionary:\n")
    print(json.dumps(todo_item, indent=2))  # Pretty print JSON

    print(f"\nAccessing specific data: Title = {todo_item['title']}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred while making the request: {e}")
except json.JSONDecodeError:
    print("Error: The response was not valid JSON.")

print("\n--- Example 2: GET Request with Parameters ---")

# Example 2: GET request with query parameters (e.g., filter todos by userId and completed status)
url_filter_todos = "https://jsonplaceholder.typicode.com/todos"
my_parameters = {
    "userId": 2,
    "completed": True  # Only get completed tasks for userId 2
}

print(f"Trying to get data from: {url_filter_todos} with filters: {my_parameters}")

try:
    response_filtered = requests.get(url_filter_todos, params=my_parameters)
    print(f"Status Code: {response_filtered.status_code}")

    filtered_todos = response_filtered.json()
    print(f"\nFound {len(filtered_todos)} completed todos for userId 2.")

    if filtered_todos:
        print("First completed todo title:", filtered_todos[0]['title'])

except requests.exceptions.RequestException as e:
    print(f"An error occurred while making the filtered request: {e}")
except json.JSONDecodeError:
    print("Error: The filtered response was not valid JSON.")

--- Example 1: Basic GET Request ---
Trying to get data from: https://jsonplaceholder.typicode.com/todos/1
Status Code: 200

Response as Python Dictionary:

{
  "userId": 1,
  "id": 1,
  "title": "delectus aut autem",
  "completed": false
}

Accessing specific data: Title = delectus aut autem

--- Example 2: GET Request with Parameters ---
Trying to get data from: https://jsonplaceholder.typicode.com/todos with filters: {'userId': 2, 'completed': True}
Status Code: 200

Found 0 completed todos for userId 2.


In [7]:
import requests
import json

print("\n--- Basic POST Requests ---")

# URL for the test API (JSONPlaceholder)
url_create_post = "https://jsonplaceholder.typicode.com/posts"

# Data you want to send (like creating a new blog post)
new_data_for_post = {
    "title": "New Sensor Data Log",
    "body": "Daily temperature readings recorded.",
    "userId": 10
}

print(f"Sending new data to create a post at: {url_create_post}")
print(f"Data to send: {new_data_for_post}")

try:
    # Send POST request (json= handles automatic conversion to JSON string)
    response_new_post = requests.post(url_create_post, json=new_data_for_post)

    print(f"Status Code: {response_new_post.status_code}")  # 201 = Created

    # Convert the JSON response back to Python dict and pretty print
    created_item = response_new_post.json()
    print(f"\nResponse from server (created item):\n{json.dumps(created_item, indent=2)}")

    # Accessing a specific part of the response
    print(f"\n✅ The new post was assigned ID: {created_item['id']}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the POST request: {e}")


--- Basic POST Requests ---
Sending new data to create a post at: https://jsonplaceholder.typicode.com/posts
Data to send: {'title': 'New Sensor Data Log', 'body': 'Daily temperature readings recorded.', 'userId': 10}
Status Code: 201

Response from server (created item):
{
  "title": "New Sensor Data Log",
  "body": "Daily temperature readings recorded.",
  "userId": 10,
  "id": 101
}

✅ The new post was assigned ID: 101


In [8]:
import requests
import json
import pandas as pd

print("\n--- Simplified Lightweight Data Puller Function with CSV Save ---")

def get_simple_weather(latitude: float, longitude: float, location_name: str) -> pd.DataFrame or None:
    """
    Fetches current weather and saves it as a CSV file.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
        location_name: Name used to save the CSV file

    Returns:
        A DataFrame with weather info or None on error
    """
    api_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": True,
        "temperature_unit": "celsius",
        "timezone": "auto"
    }

    print(f"\nPulling weather for {location_name} (Lat: {latitude}, Lon: {longitude})...")
    try:
        response = requests.get(api_url, params=params, timeout=5)
        response.raise_for_status()

        weather_data = response.json()

        if "current_weather" in weather_data:
            current = weather_data["current_weather"]
            df_weather = pd.DataFrame({
                "Time": [current['time']],
                "Temperature_C": [current['temperature']],
                "WindSpeed_kmh": [current['windspeed']]
            })
            # Save to CSV
            filename = f"{location_name.lower().replace(' ', '_')}_weather.csv"
            df_weather.to_csv(filename, index=False)
            print(f"✅ Weather data for {location_name} saved to '{filename}'")
            return df_weather
        else:
            print("⚠️ 'current_weather' not found in API response.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return None
    except json.JSONDecodeError:
        print("❌ Invalid JSON received from API.")
        return None

# Bengaluru
bengaluru_weather = get_simple_weather(12.9716, 77.5946, "Bengaluru")
if bengaluru_weather is not None:
    print("\nBengaluru Weather:\n", bengaluru_weather)

# Sriharikota
sriharikota_weather = get_simple_weather(13.6589, 80.2297, "Sriharikota")
if sriharikota_weather is not None:
    print("\nSriharikota Weather:\n", sriharikota_weather)


--- Simplified Lightweight Data Puller Function with CSV Save ---

Pulling weather for Bengaluru (Lat: 12.9716, Lon: 77.5946)...
✅ Weather data for Bengaluru saved to 'bengaluru_weather.csv'

Bengaluru Weather:
                Time  Temperature_C  WindSpeed_kmh
0  2025-06-27T02:30           21.1            9.0

Pulling weather for Sriharikota (Lat: 13.6589, Lon: 80.2297)...
✅ Weather data for Sriharikota saved to 'sriharikota_weather.csv'

Sriharikota Weather:
                Time  Temperature_C  WindSpeed_kmh
0  2025-06-27T02:30           28.2            9.2
